# Restaurant Name

In [5]:
# %pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2
# %pip install huggingface_hub
# %pip install --upgrade --quiet  huggingface_hub
# %pip install huggingface_hub
# %pip install transformers
# %pip install sentence-transformers
# %pip install sentence-transformers
# %pip install tqdm
# %pip install sentence-transformers

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached marshmallow-3.20.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached anyio-4.2.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/83.5 kB ? eta -:--:--
     ---------------------------------------  81.9/83.5 kB 4.5 MB/s eta 0:00:01
     ---------------------------------------- 83.5/83.5 kB 1.6 MB/s eta 0:00:00
  Using cached

In [2]:
%pip install --quiet python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load env variables
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
%pip install --quiet langchain_community
%pip install --quiet huggingface_hub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_community.llms import HuggingFaceHub

repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id
)

In [13]:
%pip install --quiet langchain

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain.prompts import PromptTemplate

restaurant_name_prompt_template = PromptTemplate(
    input_variables=["cuisine"],
    template= "I want to open a restaurant for {cuisine} cuisine. Suggest a good restaurant name."
)

restaurant_name_prompt_template_formatted = restaurant_name_prompt_template.format(cuisine="Chinese")
restaurant_name_prompt_template_formatted

'I want to open a restaurant for Chinese cuisine. Suggest a good restaurant name.'

In [15]:
from langchain.chains import LLMChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm
)

name_of_restaurant = restaurant_name_chain.invoke("Chinese")
name_of_restaurant

{'cuisine': 'Chinese', 'text': 'The Dragon'}

# Simple Seq Chain

In [16]:
food_items_prompt_template = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest three items to include in a menu for a restaurant called {restaurant_name}. Return the items as comma separated list"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True
)

In [18]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[restaurant_name_chain, food_items_chain], verbose=True
)

menu_items = overall_chain.invoke("Chinese")
print(f"{menu_items['input']}:\n{menu_items['output']}")



> Entering new SimpleSequentialChain chain...
The Dragon


> Entering new LLMChain chain...
Prompt after formatting:
Suggest three items to include in a menu for a restaurant called The Dragon. Return the items as comma separated list

> Finished chain.
burger, ribs, fried chicken

> Finished chain.
Chinese:
burger, ribs, fried chicken


# Seq Chain

In [34]:
from langchain.chains import SequentialChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm, output_key="restaurant_name"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True, output_key="food_items"
)

seq_chain = SequentialChain(chains=[restaurant_name_chain, food_items_chain], input_variables=["cuisine"], output_variables=["restaurant_name", "food_items"], verbose=True)

seq_chain({"cuisine": "American"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Suggest three items to include in a menu for a restaurant called The American. Return the items as comma separated list

> Finished chain.

> Finished chain.


{'cuisine': 'American',
 'restaurant_name': 'The American',
 'food_items': 'burger, ribs, steak'}